<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo I: Regresión Logística para Análisis de Sentimientos

## Importar librerías y funciones

### Prepara los datos
* `twitter_samples` contiene subconjuntos de 5,000 tweets positivos, 5,000 tweets negativos y el conjunto completo de 10,000 tweets.
     * Si utiliza los tres conjuntos de datos, introduciríamos duplicados de los tweets positivos y negativos.
     * Seleccionará solo los cinco mil tweets positivos y los cinco mil tweets negativos.

* Train test split: 20% para test, y 80% para train.


* Creear una matriz de etiquetas positivas y negativas.

### Procesamiento del tweet

La función dada `process_tweet ()` tokeniza el tweet en palabras individuales, elimina las palabras vacías y aplica la derivación.

##  Extrayendo las características

* Dada una lista de tweets, extraiga las características y guárdelas en una matriz. Extraerás dos características.
     * La primera característica es la cantidad de palabras positivas en un tweet.
     * La segunda característica es la cantidad de palabras negativas en un tweet.
* Luego entrene su clasificador de regresión logística en estas características.
* Pruebe el clasificador en un conjunto de validación. 

### Implementación de la función de extract_features.
* Esta función admite un solo tweet.
* Procesaremos el tweet usando la función `process_tweet()` importada y la guardaremos en la lista de palabras del tweet.
* Recorreremos cada palabra en la lista de palabras procesadas
     * Para cada palabra, consultaremos el diccionario `freqs` para el recuento cuando esa palabra tiene una etiqueta positiva '1'. (con clave (palabra, 1.0)
     * Hacemos lo mismo con el recuento para cuando la palabra esté asociada con la etiqueta negativa '0'. (con la clave (palabra, 0.0).)


# Implementación de la Regresión Logística


### Función sigmoide
Aprenderá a utilizar la regresión logística para la clasificación de texto.
* La función sigmoidea se define como:

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

Asigna la entrada 'z' a un valor que varía entre 0 y 1, por lo que puede tratarse como una probabilidad.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='Figures/sigmoid_plot.jpg' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /> </div>

### Logistic regression: regression y función sigmoide

La regresión logística toma una regresión lineal regular y aplica un sigmoide a la salida de la regresión lineal.

Regresion:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Tenga en cuenta que los valores $ \theta $ son "pesos". Si realizó la especialización en aprendizaje profundo, nos referimos a los pesos con el vector `w`. En este curso, usamos una variable diferente $ \theta $ para referirnos a los pesos.

Regresión logística
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Nos referiremos a 'z' como los 'logits'.

### Función de costo y gradiente

La función de costo utilizada para la regresión logística es el promedio de la pérdida de registro en todos los ejemplos de entrenamiento:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ es la cantidad de ejemplos de entrenamiento
* $y^{(i)}$ es la etiqueta real del i-ésimo dato de entrenamiento.
* $h(z(\theta)^{(i)})$ es la predicción del modelo para el i-ésimo ejemplo de entrenamiento.

La función de pérdida para un solo ejemplo de entrenamiento es
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Todos los valores de $ h $ están entre 0 y 1, por lo que los registros serán negativos. Esa es la razón del factor -1 aplicado a la suma de los dos términos de pérdida.
* Tenga en cuenta que cuando el modelo predice 1 ($ h (z (\theta)) = 1 $) y la etiqueta $ y $ también es 1, la pérdida para ese ejemplo de entrenamiento es 0.
* De manera similar, cuando el modelo predice 0 ($ h (z (\theta)) = 0 $) y la etiqueta real también es 0, la pérdida para ese ejemplo de entrenamiento es 0.
* Sin embargo, cuando la predicción del modelo es cercana a 1 ($ h (z (\theta)) = 0.9999 $) y la etiqueta es 0, el segundo término de la pérdida logarítmica se convierte en un gran número negativo, que luego se multiplica por el factor general de -1 para convertirlo en un valor de pérdida positivo. $ -1 \times (1 - 0) \times log (1 - 0.9999) \approx 9.2 $ Cuanto más se acerque la predicción del modelo a 1, mayor será la pérdida.

* Del mismo modo, si el modelo predice cerca de 0 ($ h (z) = 0.0001 $) pero la etiqueta real es 1, el primer término en la función de pérdida se convierte en un número grande: $ -1 \times log (0.0001) \approx 9.2 $. Cuanto más cercana sea la predicción a cero, mayor será la pérdida.

#### Actualizar los pesos

Para actualizar su vector de peso $ \theta $, aplicará el descenso de gradiente para mejorar iterativamente las predicciones de su modelo.
El gradiente de la función de costo $ J $ con respecto a uno de los pesos $ \theta_j $ es:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x_j \tag{5}$$
* 'i' es el índice de todos los ejemplos de formación "m".
* 'j' es el índice del peso $ \theta_j $, entonces $ x_j $ es la característica asociada con el peso $ \theta_j $

* Para actualizar el peso $ \theta_j $, lo ajustamos restando una fracción del gradiente determinado por $ \alpha $:
$$ \theta_j = \theta_j - \alpha \times \nabla_{\theta_j} J (\theta) $$
* La tasa de aprendizaje $ \alpha $ es un valor que elegimos para controlar qué tan grande será una sola actualización.


## Implementación de la función Gradiente Descendente
* El número de iteraciones `num_iters` es el número de veces que utilizará todo el conjunto de entrenamiento.
* Para cada iteración, calculará la función de costo usando todos los ejemplos de entrenamiento (hay ejemplos de entrenamiento `m`), y para todas las funciones.
* En lugar de actualizar un solo peso $ \theta_i $ a la vez, podemos actualizar todos los pesos en el vector de columna:  
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$
* $ \mathbf {\theta} $ tiene dimensiones (n + 1, 1), donde 'n' es el número de características, y hay un elemento más para el término de sesgo $ \theta_0 $ (tenga en cuenta que el valor de característica correspondiente $ \mathbf {x_0} $ es 1).
* Los 'logits', 'z', se calculan multiplicando la matriz de características 'x' con el vector de peso 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ has dimensions (m, n+1) 
    * $\mathbf{\theta}$: has dimensions (n+1, 1)
    * $\mathbf{z}$: has dimensions (m, 1)
* La predicción 'h' se calcula aplicando el sigmoide a cada elemento en 'z': $ h (z) = sigmoid (z) $, y tiene dimensiones (m, 1).
* La función de costo $ J $ se calcula tomando el producto escalar de los vectores 'y' y 'log (h)'. Dado que tanto 'y' como 'h' son vectores de columna (m, 1), transponga el vector a la izquierda, de modo que la multiplicación de matrices de un vector de fila con un vector de columna realice el producto escalar.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* La actualización de theta también está vectorizada. Debido a que las dimensiones de $ \mathbf {x} $ son (m, n + 1), y tanto $ \mathbf {h} $ como $ \mathbf {y} $ son (m, 1), necesitamos transponer $ \mathbf {x} $ y colóquelo a la izquierda para realizar la multiplicación de matrices, que luego da la respuesta (n + 1, 1) que necesitamos:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$